# Run all cells in this section

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
import torch
from torch.utils.data import Dataset

class StoryDataset(Dataset):
    def __init__(self,text):
        self.text = text
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self,idx):
        return self.text[idx]

In [5]:
from transformers import AdamW
from tqdm import tqdm
import re

class StoryGenerator:
    def __init__(self,tokenizer=None,model=None,optimizer=None,loss=None,alt_sent_gen_enabled=False):
        self.tokenizer = tokenizer
        if tokenizer == None:
            self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            
        self.model = model
        if model == None:
            self.model = GPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=self.tokenizer.eos_token_id)
        
        self.optimizer = optimizer
        if optimizer == None:
            self.optimizer = AdamW(self.model.parameters(),lr=1e-5)
            
        self.loss = loss
        if loss == None:
            self.loss = torch.nn.CrossEntropyLoss()
            
        self.alt_sent_gen_enabled = alt_sent_gen_enabled
        
    def generate_sentences(self,sent,sent_end_symbols=".?!",max_len=64,num_sentences=1):
        for i in range(0,num_sentences):
            sent += self.generate_sentence(sent=sent,sent_end_symbols=sent_end_symbols,max_len=max_len)
        return sent
        
    def generate_sentence(self,sent,sent_end_symbols=".?!",max_len=64):
        if self.alt_sent_gen_enabled:
            return self.alt_sent_gen(sent=sent)
        return self.sent_gen(sent=sent)
    
    def sent_gen(self,sent,sent_end_symbols=".?!",max_len=64):
        end_symbols = re.compile('['+ sent_end_symbols + ']')
        
        sent_len = 0
        decoded_output = sent
        end_symbol = None
        start_pos = len(decoded_output)
        while end_symbol is None or (sent_len < max_len and end_symbols.match(end_symbol) is None):
            input_ids = self.tokenizer.encode(decoded_output, return_tensors="pt")
            output_length = input_ids.size()[1]+1
            output = self.model.generate(input_ids, min_length=output_length,max_length=output_length, num_beams=3, do_sample=True, repetition_penalty=4.0)
            decoded_output = self.tokenizer.decode(output[0])
            end_symbol = decoded_output[len(decoded_output)-1]
            #right now just add 1 for every token added
            sent_len = sent_len + 1 
        return decoded_output[start_pos:]
    
    def alt_sent_gen(self,sent,sent_end_symbols=".?!",max_len=64):
        end_of_sent = len(sent)-1
        
        decoded_output = sent
        input_ids = self.tokenizer.encode(decoded_output, return_tensors="pt")
        output = self.model.generate(input_ids, max_length=max_len, num_beams=3, do_sample=True, early_stopping=True, repetition_penalty=4.0)
        decoded_output = self.tokenizer.decode(output[0])
        
        all_new_additions = decoded_output[end_of_sent+1:]
        symbol_first_index = -1
        for symbol in sent_end_symbols:
            try:
                new_symbol_first_index = all_new_additions.index(symbol)
                if symbol_first_index == -1 or new_symbol_first_index < symbol_first_index:
                    symbol_first_index = new_symbol_first_index
            except ValueError:
                pass
        if symbol_first_index < 0:
            self.alt_sent_gen(sent=sent)
        new_sentence = all_new_additions[:symbol_first_index+1]
        return new_sentence
    
    def fine_tune(self,story_loader):
        for story_batch in tqdm(story_loader):
            inputs = self.tokenizer(story_batch,padding=True,truncation=True,return_tensors="pt")
            outputs = self.model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            print(loss)
            loss.backward()
            self.optimizer.step()

In [6]:
def get_sent_embedding(sent,story_generator=None):
    #returns a list of word embeddings
    if story_generator is None: 
        return None
    if len(sent) == 0:
        return ""
    sent_embedding = []
    words = get_tokenized_sent(sent,story_generator)
    for word in words:
#         sent_embedding.append(get_word_embedding(word,story_generator))
        sent_embedding += get_word_embedding(word,story_generator)
    return sent_embedding

In [7]:
def get_tokenized_sent(sent,story_generator=None):
    #returns a list of tokens/words tokenized by gpt-2 tokenizer
    if story_generator is None: 
        return None
    if len(sent) == 0:
        return ""
    tokenized_sent = []
#     tokenized_sent = tf.Tensor([],dtype=tf.int64)
    encoded_sent = story_generator.tokenizer.encode(sent)
    for encoded_word in encoded_sent:
        tokenized_sent.append(story_generator.tokenizer.decode(encoded_word))
    return tokenized_sent

In [8]:
import numpy as np
def get_word_embedding(word,story_generator=None):
    #returns the gpt-2 embedding for a word
    if len(word) == 0 or story_generator is None:
        return None
    w_encoded = story_generator.tokenizer(word)['input_ids']
    w_tensor = torch.LongTensor(w_encoded)
    return story_generator.model.transformer.wte(w_tensor).data.numpy().tolist()

# Don't run cells in this section

In [214]:
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token = tokenizer.eos_token,padding_side="right")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token = "<|endoftext|>",padding_side="right")
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

In [9]:
import pickle

In [10]:
story_generator = StoryGenerator(tokenizer=tokenizer,model=model)
# story_generator = pickle.load(open("trained_generator.pickle","rb"))

In [11]:
 stories_file = open("stories.csv","r",encoding="utf8")

In [12]:
training_story_count = 4
training_stories = [[""] * i for i in range(0,training_story_count+1)]

curr_story = ""
prev_story = ""
story_num = -1

stories = stories_file.readlines()
for line in stories:
    if story_num < training_story_count:
        if line.find("START OF THIS PROJECT GUTENBERG EBOOK") != -1:
            if story_num+1 != 0:
                print("Loaded story " + str(story_num) + "...")
            story_num = story_num + 1
        elif line != "\n":            
            training_stories[story_num].append(line.strip())
    else:
        break
        
training_stories.pop()
stories_file.close()

Loaded story 0...
Loaded story 1...
Loaded story 2...
Loaded story 3...


In [13]:
import statistics
import copy
from copy import deepcopy
mean_line_len = statistics.mean([len(line) for line in training_stories[0]])
print(mean_line_len)
cohesion_training_stories = copy.deepcopy(training_stories)
cohesion_text_len = int(2 * mean_line_len)

56.28947368421053


In [14]:
cohesion_training_text = []
cohesion_training_labels = []
for story in cohesion_training_stories[:training_story_count]:
    cohesion_training_sents = []
    curr_len = 0
    for sent in story:
        if curr_len >= cohesion_text_len:
            cohesion_training_text.append(cohesion_training_sents)
            cohesion_training_labels.append(1)
            cohesion_training_sents = []
            curr_len = 0
        sent_tokenized = get_sent_embedding(sent,story_generator)
        if len(sent_tokenized) + curr_len <= cohesion_text_len:
            cohesion_training_sents += sent_tokenized
            curr_len += len(sent_tokenized) 
        else:
            for word in sent_tokenized:
                if curr_len + 1 <= cohesion_text_len:
                    cohesion_training_sents.append(word)
                    curr_len += 1
                else:
                    break

In [15]:
pickle_cohesion_training_text = open("cohesion_training_text.pickle","wb")
pickle.dump(cohesion_training_text,pickle_cohesion_training_text)
pickle_cohesion_training_text.close()

# cohesion_training_stories = pickle.load(open("cohesion_training_text.pickle","rb"))

In [16]:
from random import shuffle

neg_cohesion_train_txt = []
neg_cohesion_train_labels = []
for story in cohesion_training_stories[:training_story_count]:
    neg_cohesion_train_sents = []
    curr_len = 0
    for sent in story:
        if curr_len >= cohesion_text_len:
            neg_cohesion_train_txt.append(neg_cohesion_train_sents)
            neg_cohesion_train_labels.append(0)
            neg_cohesion_train_sents = []
            curr_len = 0
        sent_tokenized = get_sent_embedding(sent,story_generator)
        shuffle(sent_tokenized)
        if len(sent_tokenized) + curr_len <= cohesion_text_len:
            neg_cohesion_train_sents += sent_tokenized
            curr_len += len(sent_tokenized) 
        else:
            for word in sent_tokenized:
                if curr_len + 1 <= cohesion_text_len:
                    neg_cohesion_train_sents.append(word)
                    curr_len += 1
                else:
                    break

In [17]:
pickle_neg_cohesion_train_txt = open("neg_cohesion_train_txt.pickle","wb")
pickle.dump(neg_cohesion_train_txt,pickle_neg_cohesion_train_txt)
pickle_neg_cohesion_train_txt.close()

# neg_cohesion_train_txt = pickle.load(open("neg_cohesion_train_txt.pickle","rb"))

In [18]:
line_batch_size = 8

curr_batch_size = 0
for i,story in enumerate(training_stories):
    batched_story = [""]
    batch_num = 0
    for line in story:
        if curr_batch_size < line_batch_size:
            batched_story[batch_num] += line
            curr_batch_size += 1
        else:
            batched_story.append(line)
            batch_num += 1
            curr_batch_size = 0
    training_stories[i] = batched_story
            

In [19]:
pickle_training_batched_stories = open("training_batched_stories.pickle","wb")
pickle.dump(training_stories,pickle_training_batched_stories)
pickle_training_batched_stories.close()

# training_stories = pickle.load(open("batched_training_stories.pickle","rb"))

In [20]:
for i,batched_story in enumerate(training_stories):
    print("Story " + str(i) + " num batches: " + str(len(batched_story)))

Story 0 num batches: 55
Story 1 num batches: 75
Story 2 num batches: 468
Story 3 num batches: 61


In [230]:
from torch.utils.data import DataLoader

for story in tqdm(training_stories[:training_story_count]):
    story_dataset = StoryDataset(text=story)
    story_loader = DataLoader(story_dataset, batch_size=8)
    pickle_trained_model.close()
    

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

tensor(5.1517, grad_fn=<NllLossBackward>)



 14%|████████████                                                                        | 1/7 [00:29<02:55, 29.33s/it]

tensor(5.2697, grad_fn=<NllLossBackward>)



 29%|████████████████████████                                                            | 2/7 [01:12<03:06, 37.33s/it]

...


 75%|███████████████████████████████████████████████████████████████                     | 6/8 [02:13<00:28, 14.32s/it]

tensor(4.5829, grad_fn=<NllLossBackward>)



 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [02:23<00:12, 12.89s/it]

tensor(3.9135, grad_fn=<NllLossBackward>)



100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [16:28<00:00, 247.07s/it]


In [231]:
# pickle_trained_generator = open("trained_generator.pickle","wb")
# pickle.dump(story_generator,pickle_trained_generator)
# pickle_trained_generator.close()

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [73]:
print(cohesion_text_len)

112


In [69]:
embeddings_dim = story_generator.model.transformer.wte.embedding_dim
cohesion_model = Sequential()
# cohesion_model.add(layers.Embedding(input_dim=embeddings_dim,output_dim=64))
cohesion_model.add(layers.SimpleRNN(input_shape=(cohesion_text_len,embeddings_dim),units=64))
# cohesion_model.add(layers.SimpleRNN(input_shape=(cohesion_text_len),units=64))
cohesion_model.add(layers.Dense(32, activation='relu'))
cohesion_model.add(layers.Dense(16, activation='sigmoid'))
cohesion_model.add(layers.Dense(8, activation='relu'))
cohesion_model.add(layers.Dense(1, activation='softmax'))
cohesion_model.summary()
cohesion_model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_15 (SimpleRNN)    (None, 64)                53312     
_________________________________________________________________
dense_60 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_61 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_62 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 9         
Total params: 56,065
Trainable params: 56,065
Non-trainable params: 0
_________________________________________________________________


In [90]:
# train_data_x = cohesion_training_text + neg_cohesion_train_txt
# train_data_y = cohesion_training_labels + neg_cohesion_train_labels

In [94]:
import random

train_data_x = []
train_data_y = []
for i in range(0,100):
    rand_sample = random.randint(0,len(cohesion_training_text)-1)
    train_data_x.append(cohesion_training_text[rand_sample])
    train_data_x.append(neg_cohesion_train_txt[rand_sample])
    train_data_y.append(cohesion_training_labels[rand_sample])
    train_data_y.append(neg_cohesion_train_labels[rand_sample])

In [96]:
print(len(train_data_x))
print(len(train_data_y))

200
200


In [98]:
cohesion_model.fit(x=train_data_x,y=train_data_y,batch_size=64,epochs=10)

Epoch 1/10
4/4 [==============================] - 0s 34ms/step - loss: 0.4655 - accuracy: 0.9250
Epoch 2/10
4/4 [==============================] - 0s 30ms/step - loss: 0.3928 - accuracy: 0.9600
Epoch 3/10
4/4 [==============================] - 0s 37ms/step - loss: 0.3278 - accuracy: 0.9750
Epoch 4/10
4/4 [==============================] - 0s 34ms/step - loss: 0.2754 - accuracy: 0.9850
Epoch 5/10
4/4 [==============================] - 0s 40ms/step - loss: 0.2320 - accuracy: 0.9850
Epoch 6/10
4/4 [==============================] - 0s 32ms/step - loss: 0.1946 - accuracy: 0.9900
Epoch 7/10
4/4 [==============================] - 0s 33ms/step - loss: 0.1606 - accuracy: 0.9950
Epoch 8/10
4/4 [==============================] - 0s 34ms/step - loss: 0.1361 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 0s 33ms/step - loss: 0.1196 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 0s 32ms/step - loss: 0.1061 - accuracy: 1.0000


In [99]:
cohesion_model.save("cohesion_model")
# cohesion_model = tf.keras.models.load_model("cohesion_model")

INFO:tensorflow:Assets written to: cohesion_model\assets


In [100]:
cohesion_model.predict([cohesion_training_text[1]])

array([[0.9153782]], dtype=float32)

# Run all cells in this section

In [10]:
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential

cohesion_text_len = 112
story_generator = pickle.load(open("trained_generator.pickle","rb"))
embeddings_dim = embeddings_dim = story_generator.model.transformer.wte.embedding_dim
# cohesion_training_stories = pickle.load(open("cohesion_training_text.pickle","rb"))
# neg_cohesion_train_txt = pickle.load(open("neg_cohesion_train_txt.pickle","rb"))
# training_stories = pickle.load(open("batched_training_stories.pickle","rb"))
cohesion_model = tf.keras.models.load_model("cohesion_model")

In [70]:
def get_cohesion_value(text,story_generator,cohesion_model,block_len=112):
    # text parameter is a string
    # This section of code groups the story text into blocks of
    # 112 word embeddings (average words per line * 2) 
    cohesion_blocks = []
    text_as_lines = text.splitlines()
    
    cohesion_usr_sents = []
    curr_len = 0
    for line in text_as_lines:
        if curr_len >= block_len:
            cohesion_blocks.append(cohesion_usr_sents)
            cohesion_usr_sents = []
            curr_len = 0
        line_tokenized = get_sent_embedding(line,story_generator)
        if len(line_tokenized) + curr_len <= block_len:
            cohesion_usr_sents += line_tokenized
            curr_len += len(line_tokenized) 
        else:
            for word in line_tokenized:
                if curr_len + 1 <= block_len:
                    cohesion_usr_sents.append(word)
                    curr_len += 1
                else:
                    break
    if len(cohesion_blocks) == 0:
        raise RuntimeError("block length too small, try using a larger text or lowering block__len")
        
    # This section of code uses the cohesion_model (cohesion RNN) to
    # classify the text and return the likelyhood that the text is cohesive
    raw_predictions = cohesion_model.predict(cohesion_blocks)
    return raw_predictions[0][0]

# Don't run cells in this section

In [75]:
test_story = "I am already far north of London, and as I walk in the streets of \n\
Petersburgh, I feel a cold northern breeze play upon my cheeks, which \n \
braces my nerves and fills me with delight. Do you understand this \n\
feeling?\n"

print(get_cohesion_value(test_story,story_generator,cohesion_model,block_len=20))

0.8917127


In [87]:
test_story = "I far already London dog of north am, I as and the in walk of streets \n\
Peteear burgh, a el brether cold cat m play which cheeks, \n \
nervbra wolf m fills and delight with. rstand pig you Do this \n\
feeling?\n"

print(get_cohesion_value(test_story,story_generator,cohesion_model,block_len=20))

0.9098319


# Run these cells to generate and evaluate a story

In [99]:
user_sent = input("Enter the first few sentences of your story as a prompt: ")
user_story_len = input("Enter the length of your story (number of sentences): ")
story_generator.alt_sent_gen_enabled = False
story = story_generator.generate_sentences(sent=user_sent,num_sentences=int(user_story_len))
print(story)
print()

Enter the first few sentences of your story as a prompt: I am already far north of London, and as I walk in the streets of Petersburgh, I feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. Do you understand this feeling?
Enter the length of your story (number of sentences): 20
I am already far north of London, and as I walk in the streets of Petersburgh, I feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. Do you understand this feeling? It is not just that it was always there; it has been here for years now. The first thing to do was to make sure that she had no other choice but to go on her own way. She did not want to be alone: she wanted to get out of the house by herself. But why should she? Why should she stay at home? And what else could she do? What else would she do? There were three reasons--the one being that she didn't know how to keep up with the new arrivals or who they were go

In [131]:
import nltk
story_as_lines = nltk.sent_tokenize(story)
story_with_lines = '\n'.join(story_as_lines)
print(get_cohesion_value(text=story_with_lines,story_generator=story_generator,cohesion_model=cohesion_model,block_len=20))

0.88715446
